# Travel Intent Classification & NER - Google Colab GPU

Ce notebook Colab (optimisé pour **GPU**) utilise **Transformers** pour du **Fine-Tuning** sur :
1. **Intent Classification** : Classification en 4 classes (TRIP, NOT_TRIP, UNKNOWN, NOT_FRENCH)
2. **Named Entity Recognition (NER)** : Extraction de Departure et Destination

**Dataset** : Chargé depuis Google Drive (`/content/drive/MyDrive/nlp_dataset/`)

**Modèle** : CamemBERT (modèle français pré-entraîné)

In [ ]:
print('Travel Order Resolver - Google Colab GPU + Transformers Fine-Tuning')
print('='*70)

## 1 - Vérification du GPU et Installation des dépendances

Vérifie la disponibilité du GPU et installe les librairies nécessaires :
- `transformers` : Pour CamemBERT et le fine-tuning
- `datasets` : Pour gérer les datasets
- `evaluate` : Pour les métriques d'évaluation
- `seqeval` : Pour les métriques NER
- `accelerate` : Pour l'optimisation GPU

In [ ]:
# Vérifier la disponibilité du GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Pas de GPU détecté. Allez dans Runtime > Change runtime type > GPU")

# Installation des packages
!pip install -q transformers datasets evaluate seqeval accelerate
print("Installation terminée!")

## 2 - Montage de Google Drive et Chargement des Datasets

Monte Google Drive pour accéder aux datasets depuis `/content/drive/MyDrive/nlp_dataset/`

**Structure attendue dans Drive** :
```
MyDrive/
  nlp_dataset/
    train_set.csv
    test_set.csv
    cities_fr.txt (optionnel)
```

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Chemins vers les datasets dans Google Drive
import os
import pandas as pd

DATASET_DIR = '/content/drive/MyDrive/nlp_dataset'
train_csv = os.path.join(DATASET_DIR, 'train_set.csv')
test_csv = os.path.join(DATASET_DIR, 'test_set.csv')
cities_file = os.path.join(DATASET_DIR, 'cities_fr.txt')

# Vérifier l'existence des fichiers
if not os.path.exists(train_csv) or not os.path.exists(test_csv):
    print("ERREUR: Fichiers introuvables!")
    print(f"   Assurez-vous que les fichiers sont dans: {DATASET_DIR}")
    print("   - train_set.csv")
    print("   - test_set.csv")
else:
    print(" Fichiers trouvés!")
    train_df = pd.read_csv(train_csv, encoding='utf-8')
    test_df = pd.read_csv(test_csv, encoding='utf-8')
    print(f"\n Train shape: {train_df.shape}")
    print(f" Test shape: {test_df.shape}")
    print(f"\n Distribution des classes (Train):")
    print(train_df['intent'].value_counts())
    print(f"\n Aperçu du dataset:")
    display(train_df.head(3))

## 3 - Prétraitement : Parser les Entities JSON

Parse la colonne `entities` (format JSON) pour extraire les annotations Departure/Destination.

**Format des entities** :
```json
[{"start": 10, "end": 15, "label": "Departure"}, {"start": 20, "end": 25, "label": "Destination"}]
```

In [ ]:
import json

def parse_entities_field(row):
    """Parse la colonne entities (JSON) et valide les annotations."""
    try:
        ents = json.loads(row['entities'])
    except Exception:
        ents = []
    
    valid = []
    txt = row.get('text', '')
    for ent in ents:
        if 'start' in ent and 'end' in ent and 'label' in ent:
            if 0 <= ent['start'] < ent['end'] <= len(txt):
                valid.append(ent)
    return valid

# Appliquer le parsing
train_df['parsed_entities'] = train_df.apply(parse_entities_field, axis=1)
test_df['parsed_entities'] = test_df.apply(parse_entities_field, axis=1)

print("Entities parsées avec succès!")
print(f"\nExemple avec entities:")
trip_examples = train_df[train_df['intent'] == 'TRIP'].head(2)
for idx, row in trip_examples.iterrows():
    print(f"\nTexte: {row['text']}")
    print(f"Intent: {row['intent']}")
    print(f"Entities: {row['parsed_entities']}")

## 4 - Préparation des Datasets HuggingFace

Conversion des DataFrames Pandas en Datasets HuggingFace pour l'entraînement avec Transformers.

Les datasets seront préparés pour :
1. **Intent Classification** : Text → Label (TRIP, NOT_TRIP, UNKNOWN, NOT_FRENCH)
2. **NER** : Text → Tokens avec labels BIO (B-Departure, I-Departure, B-Destination, I-Destination, O)

In [ ]:
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

# === DATASET POUR INTENT CLASSIFICATION ===

# Encoder les labels d'intent
label_encoder = LabelEncoder()
all_intents = list(train_df['intent'].unique()) + list(test_df['intent'].unique())
label_encoder.fit(list(set(all_intents)))

print("Classes d'intent:")
for i, label in enumerate(label_encoder.classes_):
    print(f"   {i}: {label}")

# Créer les datasets HuggingFace
train_df['label'] = label_encoder.transform(train_df['intent'])
test_df['label'] = label_encoder.transform(test_df['intent'])

intent_train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
intent_test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

print(f"\nIntent datasets créés:")
print(f"   Train: {len(intent_train_dataset)} exemples")
print(f"   Test: {len(intent_test_dataset)} exemples")

## 5 - Fine-Tuning Intent Classification avec CamemBERT

Entraînement d'un modèle de classification d'intent basé sur **CamemBERT** (modèle français).

**Paramètres** :
- Modèle : `camembert-base`
- Nombre d'epochs : 3
- Batch size : 16 (train) / 32 (eval)
- Learning rate : 2e-5
- Optimisation : AdamW avec weight decay

In [ ]:
# Désactiver WandB (tracking des expériences)
import os
os.environ['WANDB_DISABLED'] = 'true'
print("✅ WandB désactivé")

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate
import numpy as np

# Charger le tokenizer CamemBERT
tokenizer = AutoTokenizer.from_pretrained('camembert-base')

# Fonction de tokenization
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

# Tokenizer les datasets
tokenized_train = intent_train_dataset.map(tokenize_function, batched=True)
tokenized_test = intent_test_dataset.map(tokenize_function, batched=True)

# Charger le modèle CamemBERT pour classification
num_labels = len(label_encoder.classes_)
intent_model = AutoModelForSequenceClassification.from_pretrained(
    'camembert-base',
    num_labels=num_labels,
    id2label={i: label for i, label in enumerate(label_encoder.classes_)},
    label2id={label: i for i, label in enumerate(label_encoder.classes_)}
)

# Définir les métriques
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

def compute_intent_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')
    
    return {
        'accuracy': accuracy['accuracy'],
        'f1_macro': f1['f1']
    }

# Arguments d'entraînement (optimisés pour GPU)
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/nlp_dataset/models/intent_classifier',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1_macro',
    logging_steps=50,
    warmup_steps=100,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    report_to='none',  # Désactiver WandB et autres trackers
)

# Créer le Trainer
intent_trainer = Trainer(
    model=intent_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_intent_metrics
)

print("🚀 Début du fine-tuning Intent Classification...")
print("="*70)

# Entraîner le modèle
intent_trainer.train()

print("\n✅ Fine-tuning Intent Classification terminé!")

## 6 - Évaluation du Modèle Intent Classification

Évalue le modèle entraîné sur le test set et affiche les métriques détaillées :
- **Accuracy** : Précision globale
- **F1-Score (Macro)** : F1 moyen sur toutes les classes
- **Matrice de confusion** : Visualisation des erreurs

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Évaluer sur le test set
eval_results = intent_trainer.evaluate()

print("Résultats sur le Test Set:")
print("="*70)
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")

# Prédictions détaillées
predictions = intent_trainer.predict(tokenized_test)
predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

# Classification report
print("\nClassification Report:")
print("="*70)
print(classification_report(
    true_labels,
    predicted_labels,
    target_names=label_encoder.classes_,
    digits=4
))

# Matrice de confusion
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.title('Matrice de Confusion - Intent Classification')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.tight_layout()
plt.show()

# Sauvegarder le meilleur modèle
intent_trainer.save_model('/content/drive/MyDrive/nlp_dataset/models/intent_classifier_best')
tokenizer.save_pretrained('/content/drive/MyDrive/nlp_dataset/models/intent_classifier_best')

print("\nModèle sauvegardé dans Google Drive!")

## 7 - Préparation du Dataset NER (Token Classification)

Conversion des annotations au format **BIO** (Begin-Inside-Outside) pour le NER :
- **O** : Pas une entité
- **B-Departure** : Début du lieu de départ
- **I-Departure** : Continuation du lieu de départ
- **B-Destination** : Début du lieu de destination
- **I-Destination** : Continuation du lieu de destination

Seuls les exemples avec `intent=TRIP` ont des annotations NER.

In [ ]:
from transformers import AutoTokenizer

# Tokenizer rapide pour NER (nécessaire pour offset_mapping)
tokenizer_fast = AutoTokenizer.from_pretrained('camembert-base', use_fast=True)

# Labels NER en format BIO
ner_labels = ['O', 'B-Departure', 'I-Departure', 'B-Destination', 'I-Destination']
label2id = {label: i for i, label in enumerate(ner_labels)}
id2label = {i: label for label, i in label2id.items()}

print("Labels NER:")
for label_id, label_name in id2label.items():
    print(f"   {label_id}: {label_name}")

def convert_to_bio_tags(text, entities):
    """
    Convertit les annotations (start, end, label) en tags BIO pour chaque token.
    """
    # Tokenizer le texte
    encoding = tokenizer_fast(
        text,
        return_offsets_mapping=True,
        truncation=True,
        max_length=128,
        padding='max_length'
    )
    
    offset_mapping = encoding['offset_mapping']
    labels = ['O'] * len(offset_mapping)
    
    # Pour chaque entité, marquer les tokens correspondants
    for entity in entities:
        start_char = entity['start']
        end_char = entity['end']
        entity_label = entity['label']
        
        token_indices = []
        for idx, (token_start, token_end) in enumerate(offset_mapping):
            # Ignorer les tokens padding
            if token_start == token_end == 0:
                continue
            
            # Vérifier si le token chevauche l'entité
            if not (token_end <= start_char or token_start >= end_char):
                token_indices.append(idx)
        
        # Appliquer les tags BIO
        if token_indices:
            labels[token_indices[0]] = f'B-{entity_label}'
            for idx in token_indices[1:]:
                labels[idx] = f'I-{entity_label}'
    
    # Convertir en IDs
    label_ids = [label2id.get(label, 0) for label in labels]
    
    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label_ids
    }

# Préparer les datasets NER
print("\nConversion en format BIO...")
ner_train_data = []
for _, row in train_df.iterrows():
    entities = row['parsed_entities'] if row['intent'] == 'TRIP' else []
    ner_example = convert_to_bio_tags(row['text'], entities)
    ner_train_data.append(ner_example)

ner_test_data = []
for _, row in test_df.iterrows():
    entities = row['parsed_entities'] if row['intent'] == 'TRIP' else []
    ner_example = convert_to_bio_tags(row['text'], entities)
    ner_test_data.append(ner_example)

# Créer les datasets HuggingFace
ner_train_dataset = Dataset.from_list(ner_train_data)
ner_test_dataset = Dataset.from_list(ner_test_data)

print(f"Datasets NER créés:")
print(f"   Train: {len(ner_train_dataset)} exemples")
print(f"   Test: {len(ner_test_dataset)} exemples")

# Afficher un exemple
print(f"\nExemple de tokenization BIO:")
example_idx = train_df[train_df['intent'] == 'TRIP'].index[0]
example_text = train_df.loc[example_idx, 'text']
example_entities = train_df.loc[example_idx, 'parsed_entities']
print(f"Texte: {example_text}")
print(f"Entities: {example_entities}")

tokens = tokenizer_fast.tokenize(example_text)
bio_example = convert_to_bio_tags(example_text, example_entities)
bio_labels = [id2label[lid] for lid in bio_example['labels']]

print(f"\nTokens et labels BIO (premiers 15):")
for token, label in list(zip(tokens, bio_labels))[:15]:
    print(f"   {token:20s} → {label}")

## 8 - Fine-Tuning NER avec CamemBERT (Token Classification)

Entraînement d'un modèle de **Named Entity Recognition** basé sur CamemBERT.

**Tâche** : Identifier et extraire les lieux de départ (Departure) et de destination (Destination) dans les phrases de voyage.

**Paramètres** :
- Modèle : `camembert-base` (Token Classification)
- Nombre d'epochs : 4
- Batch size : 8 (train) / 16 (eval)
- Learning rate : 3e-5
- Métrique : F1-Score (seqeval)

In [ ]:
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
import evaluate
import numpy as np

# Charger le modèle CamemBERT pour Token Classification
ner_model = AutoModelForTokenClassification.from_pretrained(
    'camembert-base',
    num_labels=len(ner_labels),
    id2label=id2label,
    label2id=label2id
)

# Data collator pour padding dynamique
data_collator = DataCollatorForTokenClassification(tokenizer_fast)

# Charger la métrique seqeval
seqeval_metric = evaluate.load('seqeval')

def compute_ner_metrics(eval_pred):
    """
    Calcule les métriques NER (precision, recall, F1) avec seqeval.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # Convertir les prédictions et labels en listes de tags
    true_labels = []
    pred_labels = []
    
    for pred_seq, label_seq in zip(predictions, labels):
        true_tags = []
        pred_tags = []
        
        for pred_id, label_id in zip(pred_seq, label_seq):
            # Ignorer les labels de padding (-100)
            if label_id != -100:
                true_tags.append(id2label[label_id])
                pred_tags.append(id2label[pred_id])
        
        true_labels.append(true_tags)
        pred_labels.append(pred_tags)
    
    # Calculer les métriques
    results = seqeval_metric.compute(predictions=pred_labels, references=true_labels)
    
    return {
        'precision': results['overall_precision'],
        'recall': results['overall_recall'],
        'f1': results['overall_f1'],
        'accuracy': results['overall_accuracy']
    }

# Arguments d'entraînement (optimisés pour GPU)
ner_training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/nlp_dataset/models/ner_model',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    logging_steps=100,
    warmup_steps=200,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    report_to='none',  # Désactiver WandB et autres trackers
)

# Créer le Trainer
ner_trainer = Trainer(
    model=ner_model,
    args=ner_training_args,
    train_dataset=ner_train_dataset,
    eval_dataset=ner_test_dataset,
    tokenizer=tokenizer_fast,
    data_collator=data_collator,
    compute_metrics=compute_ner_metrics
)

print("🚀 Début du fine-tuning NER (Token Classification)...")
print("="*70)

# Entraîner le modèle
ner_trainer.train()

print("\n✅ Fine-tuning NER terminé!")

## 9 - Évaluation du Modèle NER

Évalue le modèle NER sur le test set avec des métriques détaillées par entité :
- **Precision** : Précision de détection des entités
- **Recall** : Taux de rappel des entités
- **F1-Score** : Score F1 global et par type d'entité (Departure, Destination)

In [ ]:
from seqeval.metrics import classification_report as seqeval_report

# Évaluer sur le test set
ner_eval_results = ner_trainer.evaluate()

print("Résultats NER sur le Test Set:")
print("="*70)
for key, value in ner_eval_results.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")

# Prédictions détaillées
ner_predictions = ner_trainer.predict(ner_test_dataset)
predicted_logits = ner_predictions.predictions
predicted_labels = np.argmax(predicted_logits, axis=-1)
true_labels = ner_predictions.label_ids

# Convertir en tags pour seqeval
true_tags_list = []
pred_tags_list = []

for pred_seq, label_seq in zip(predicted_labels, true_labels):
    true_tags = []
    pred_tags = []
    
    for pred_id, label_id in zip(pred_seq, label_seq):
        if label_id != -100:
            true_tags.append(id2label[label_id])
            pred_tags.append(id2label[pred_id])
    
    true_tags_list.append(true_tags)
    pred_tags_list.append(pred_tags)

# Classification report détaillé
print("\nClassification Report NER (par entité):")
print("="*70)
print(seqeval_report(true_tags_list, pred_tags_list, digits=4))

# Sauvegarder le meilleur modèle
ner_trainer.save_model('/content/drive/MyDrive/nlp_dataset/models/ner_model_best')
tokenizer_fast.save_pretrained('/content/drive/MyDrive/nlp_dataset/models/ner_model_best')

print("\nModèle NER sauvegardé dans Google Drive!")

# Afficher quelques exemples de prédictions
print("\nExemples de prédictions NER:")
print("="*70)

trip_test_indices = test_df[test_df['intent'] == 'TRIP'].head(5).index
for idx in trip_test_indices:
    text = test_df.loc[idx, 'text']
    true_entities = test_df.loc[idx, 'parsed_entities']
    
    # Prédire avec le modèle
    inputs = tokenizer_fast(text, return_tensors='pt', truncation=True, max_length=128)
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        ner_model.cuda()
    
    with torch.no_grad():
        outputs = ner_model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    
    # Décoder les tokens et labels
    tokens = tokenizer_fast.convert_ids_to_tokens(inputs['input_ids'][0])
    predicted_tags = [id2label[p.item()] for p in predictions[0]]
    
    print(f"\nTexte: {text}")
    print(f"Vraies entités: {true_entities}")
    print("Prédictions:")
    
    current_entity = None
    current_tokens = []
    
    for token, tag in zip(tokens, predicted_tags):
        if token in ['<s>', '</s>', '<pad>']:
            continue
        
        if tag.startswith('B-'):
            if current_entity:
                print(f"   {current_entity}: {''.join(current_tokens).replace('▁', ' ').strip()}")
            current_entity = tag[2:]
            current_tokens = [token]
        elif tag.startswith('I-') and current_entity:
            current_tokens.append(token)
        else:
            if current_entity:
                print(f"   {current_entity}: {''.join(current_tokens).replace('▁', ' ').strip()}")
                current_entity = None
                current_tokens = []
    
    if current_entity:
        print(f"   {current_entity}: {''.join(current_tokens).replace('▁', ' ').strip()}")
    
    print("-" * 70)

## 10 - Pipeline d'Inférence Complet

Combine les deux modèles (Intent Classification + NER) pour créer un pipeline d'inférence complet.

**Workflow** :
1. Prédire l'intent du message
2. Si intent = TRIP, extraire Departure et Destination avec NER
3. Retourner le résultat formaté

In [ ]:
from transformers import pipeline

# Charger les pipelines
print("Chargement des modèles...")

# Pipeline Intent Classification
intent_pipeline = pipeline(
    'text-classification',
    model='/content/drive/MyDrive/nlp_dataset/models/intent_classifier_best',
    tokenizer='/content/drive/MyDrive/nlp_dataset/models/intent_classifier_best',
    device=0 if torch.cuda.is_available() else -1
)

# Pipeline NER
ner_pipeline = pipeline(
    'token-classification',
    model='/content/drive/MyDrive/nlp_dataset/models/ner_model_best',
    tokenizer='/content/drive/MyDrive/nlp_dataset/models/ner_model_best',
    aggregation_strategy='simple',
    device=0 if torch.cuda.is_available() else -1
)

print("Pipelines chargés!")

def predict_travel_order(text):
    """
    Pipeline complet : Intent + NER
    """
    # 1. Prédire l'intent
    intent_result = intent_pipeline(text)[0]
    predicted_intent = intent_result['label']
    confidence = intent_result['score']
    
    result = {
        'text': text,
        'intent': predicted_intent,
        'confidence': confidence
    }
    
    # 2. Si TRIP, extraire les entités
    if predicted_intent == 'TRIP':
        ner_results = ner_pipeline(text)
        
        departure = None
        destination = None
        
        for entity in ner_results:
            entity_label = entity.get('entity_group', entity.get('entity'))
            word = entity['word'].replace('▁', ' ').strip()
            
            if 'Departure' in entity_label and not departure:
                departure = word
            elif 'Destination' in entity_label and not destination:
                destination = word
        
        result['departure'] = departure
        result['destination'] = destination
    
    return result

# Tester sur quelques exemples
print("\nTests sur des exemples:")
print("="*70)

test_examples = [
    "Je voudrais un billet de Cotonou à Porto-Novo demain",
    "Bonjour, un aller-retour Paris Marseille s'il vous plaît",
    "Merci pour votre email, je confirme ma présence",
    "Hello, I need a ticket to London",
    "zxqwerty azerty"
]

for i, example in enumerate(test_examples, 1):
    print(f"\n{i}. {example}")
    result = predict_travel_order(example)
    print(f"   Intent: {result['intent']} (confiance: {result['confidence']:.2%})")
    
    if result['intent'] == 'TRIP':
        dep = result.get('departure', 'N/A')
        dest = result.get('destination', 'N/A')
        print(f"   Departure: {dep}")
        print(f"   Destination: {dest}")
        print(f"   → Sortie: {i},{dep},{dest}")
    else:
        print(f"   → Sortie: {i},{result['intent']}")

print("\n" + "="*70)
print("Pipeline d'inférence prêt à l'emploi!")

## 11 - Traitement Batch d'un Fichier d'Input

Lit un fichier d'input au format `sentenceID,sentence` et génère un fichier de sortie avec les prédictions.

**Format d'entrée** : `1,Je voudrais un billet Paris Lyon`  
**Format de sortie** :
- Si TRIP : `1,Paris,Lyon`
- Sinon : `1,NOT_TRIP` (ou UNKNOWN, NOT_FRENCH)

In [ ]:
def process_batch_file(input_file, output_file):
    """
    Traite un fichier batch et génère les prédictions.
    
    Format entrée: sentenceID,sentence
    Format sortie: sentenceID,departure,destination (si TRIP) ou sentenceID,intent (sinon)
    """
    output_lines = []
    
    print(f"Lecture de {input_file}...")
    
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print(f"Traitement de {len(lines)} phrases...")
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        try:
            # Parser la ligne
            parts = line.split(',', 1)
            if len(parts) != 2:
                continue
            
            sentence_id, text = parts
            
            # Prédire
            result = predict_travel_order(text)
            
            # Formater la sortie
            if result['intent'] == 'TRIP':
                departure = result.get('departure', 'UNKNOWN')
                destination = result.get('destination', 'UNKNOWN')
                
                if departure and destination:
                    output_line = f"{sentence_id},{departure},{destination}"
                else:
                    output_line = f"{sentence_id},UNKNOWN"
            else:
                output_line = f"{sentence_id},{result['intent']}"
            
            output_lines.append(output_line)
            
        except Exception as e:
            print(f"Erreur sur la ligne: {line[:50]}... → {e}")
            continue
    
    # Sauvegarder
    print(f"\nSauvegarde de {len(output_lines)} résultats dans {output_file}...")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(output_lines))
    
    print(f"Traitement terminé!")
    return output_file

# Exemple d'utilisation
input_file_path = '/content/drive/MyDrive/nlp_dataset/sample_nlp_input.txt'
output_file_path = '/content/drive/MyDrive/nlp_dataset/predictions_output.txt'

# Vérifier si le fichier existe
if os.path.exists(input_file_path):
    result_file = process_batch_file(input_file_path, output_file_path)
    
    print(f"\nAperçu des résultats:")
    print("="*70)
    with open(result_file, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 10:  # Afficher les 10 premières lignes
                break
            print(line.strip())
    
    print("\nFichier complet sauvegardé dans Google Drive!")
else:
    print(f"Fichier d'entrée non trouvé: {input_file_path}")
    print("   Uploadez votre fichier dans Google Drive ou modifiez le chemin.")

## Récapitulatif et Instructions d'Utilisation

### Ce qui a été fait :
1. **Fine-tuning Intent Classification** : Modèle CamemBERT entraîné pour classifier 4 types d'intent (TRIP, NOT_TRIP, UNKNOWN, NOT_FRENCH)
2. **Fine-tuning NER** : Modèle CamemBERT entraîné pour extraire Departure et Destination
3. **Pipeline d'inférence** : Système complet combinant les deux modèles
4. **Traitement batch** : Fonction pour traiter des fichiers d'input en masse

### Modèles sauvegardés dans Google Drive :
- `/content/drive/MyDrive/nlp_dataset/models/intent_classifier_best/`
- `/content/drive/MyDrive/nlp_dataset/models/ner_model_best/`

### Pour réutiliser les modèles plus tard :

```python
from transformers import pipeline
import torch

# Charger le pipeline Intent
intent_pipe = pipeline(
    'text-classification',
    model='/content/drive/MyDrive/nlp_dataset/models/intent_classifier_best',
    device=0 if torch.cuda.is_available() else -1
)

# Charger le pipeline NER
ner_pipe = pipeline(
    'token-classification',
    model='/content/drive/MyDrive/nlp_dataset/models/ner_model_best',
    aggregation_strategy='simple',
    device=0 if torch.cuda.is_available() else -1
)

# Utiliser
result = intent_pipe("Je veux aller de Cotonou à Porto-Novo")
entities = ner_pipe("Je veux aller de Cotonou à Porto-Novo")
```

### Performances attendues :
- **Intent Classification** : F1-Score > 0.95 (selon la qualité du dataset)
- **NER** : F1-Score > 0.90 pour Departure et Destination

### Optimisations possibles :
1. Augmenter le nombre d'epochs pour améliorer les performances
2. Ajuster les hyperparamètres (learning rate, batch size, weight decay)
3. Essayer d'autres modèles français : FlauBERT, BARThez
4. Ajouter de l'augmentation de données
5. Utiliser des techniques d'ensemble (combiner plusieurs modèles)

---

**Notebook complété avec succès !**